In [1]:
import pycocotools
from ultralytics import YOLO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import yaml
import torch
import shutil

In [2]:
train_dir = "C:\Users\danielian\Documents\carr-detection\data\training_images"
train_labels = "C:\Users\danielian\Documents\car-detection\data\train_solution_bounding_boxes (1).csv"
test_dir = "C:\Users\danielian\Documents\car-detection\data\testing_images"

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1401578287.py, line 1)

In [ ]:
imgs_list=list(sorted(os.listdir(train_dir)))
idxs=list(range(len(imgs_list)))
np.random.shuffle(idxs)

train_idx=idxs[:int(0.8*len(idxs))]
val_idx=idxs[int(0.8*len(idxs)):]

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'data\training_images'

In [ ]:
root_dir="C:\Users\danielian\Documents\car-detection\data\working\data"
labels_dir="C:\Users\danielian\Documents\car-detection\data\working\labels"
images_dir="C:\Users\danielian\Documents\car-detection\data\working\images"

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1495942294.py, line 1)

In [ ]:
df=pd.read_csv(train_labels)

width=676
height=380

df["class"]=0
df.rename(columns={'image':'img_name'}, inplace=True)

df["x_centre"]=(df["xmin"]+df["xmax"])/2
df["y_centre"]=(df["ymin"]+df["ymax"])/2
df["width"]=(df["xmax"]-df["xmin"])
df["height"]=(df["ymax"]-df["ymin"])

#normalizing bounding box coordinates
df["x_centre"]=df["x_centre"]/width
df["y_centre"]=df["y_centre"]/height
df["width"]=df["width"]/width
df["height"]=df["height"]/height

df_yolo=df[["img_name","class","x_centre","y_centre","width","height"]]
df_yolo.head()

In [ ]:
for idx,img_name in enumerate(imgs_list):
    subset="train"
    if idx in val_idx:
        subset="val"
        
    if np.isin(img_name,df_yolo["img_name"]):
        columns=["class","x_centre","y_centre","width","height"]
        img_bbox=df_yolo[df_yolo["img_name"]==img_name][columns].values
        
        label_file_path=os.path.join(labels_dir,subset,img_name[:-4]+".txt")
        with open(label_file_path,"w+") as f:
            for row in img_bbox:
                text=" ".join(row.astype(str))
                f.write(text)
                f.write("\n")
                
    old_image_path=os.path.join(train_dir,img_name)
    new_image_path=os.path.join(images_dir,subset,img_name)
    shutil.copy(old_image_path,new_image_path)

In [ ]:
yolo_format=dict(path="C:\Users\danielian\Documents\car-detection\data\working\data",
                 train="C:\Users\danielian\Documents\car-detection\data\working\data\images\train",
                 val="C:\Users\danielian\Documents\car-detection\data\working\data\images\val",
                 nc=1,
                 names={0:"car"})
             
with open('C:\Users\danielian\Documents\car-detection\data\working\yolo.yaml', 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1169988869.py, line 1)

Training

In [ ]:
model=YOLO('yolov8m.pt')
model.train(data="C:\Users\danielian\Documents\car-detection\data\working\yolo.yaml",epochs=50,patience=5,batch=8,lr0=0.0005,imgsz=640)

Validate

In [ ]:
path_best_weights="C:\Users\danielian\Documents\car-detection\data\working\runs\detect\train\weights\best.pt"
model = YOLO(path_best_weights) 

metrics = model.val()

NameError: name 'YOLO' is not defined

In [ ]:
print(f"Mean Average Precision @.5:.95 : {metrics.box.map}")    
print(f"Mean Average Precision @ .50   : {metrics.box.map50}") 
print(f"Mean Average Precision @ .70   : {metrics.box.map75}")